In [1]:
# Import Flask
from flask import Flask, jsonify

In [2]:
#Import Dependancies
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
from sqlalchemy.engine import create_engine

In [4]:
engine = create_engine("sqlite:///../hawaii.sqlite")

In [5]:
# Reflect Existing Database Into a New Model
Base = automap_base()
# Reflect the Tables
Base.prepare(engine, reflect=True)

In [6]:
session = Session(engine)

In [7]:
# Flask Setup
app = Flask(__name__)

In [ ]:
# Save References to Each Table
measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
# Flask Route Setup
@app.route("/")
def main():
    """Lists all available routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/start<br/>"
        f"/api/v1.0/start/end"
    )

In [9]:
# Create Precipitation Route
@app.route("/api/v1.0/precipitation")
def precipitation():
        previous_year = dt.date(2017,8,23) - dt.timedelta(days=365)
        # Design a Query to Retrieve the Last 12 Months of Precipitation Data Selecting Only the `date` and `prcp` Values
        previous_year_prcp = session.query(Measurement.date, Measurement.prcp).\
                filter(Measurement.date >= previous_year).\
                order_by(Measurement.date).all()
        session.close()
        # Convert  Into a Dictionary
        previous_year_prcp_list = dict(previous_year_prcp)
        return jsonify(previous_year_prcp_list)
        

In [10]:
# Create Station Route
@app.route("/api/v1.0/stations")
def stations():
        # Return JSON List of Stations 
        all_stations = session.query(Station.station, Station.name).all()
        session.close()
        # Unravel results into a 1D array and convert to a list
        station_list=list(np.ravel(all_stations))
        return jsonify(station_list)

In [13]:
# Create TOBs Route
@app.route("/api/v1.0/tobs")
def tobs():
#Query the primary station for all tobs from the last year
 prev_year = dt.date(2017,8,23) - dt.timedelta(days=365)
 results = session.query(Measurement.tobs).\
       filter(Measurement.station == 'USC00519281').\
       filter(Measurement.date >= prev_year).all()
   # Unravel results into a 1D array and convert to a list
 temps = list(np.ravel(results))
   # Return the results
 return jsonify(temps=temps)

In [14]:
@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def stats(start=None, end=None):
   """Return TMIN, TAVG, TMAX."""
   # Select statement
   sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
   if not end:
       # calculate TMIN, TAVG, TMAX for dates greater than start
       results = session.query(*sel).\
           filter(Measurement.date >= start).all()
       # Unravel results into a 1D array and convert to a list
       temps = list(np.ravel(results))
       return jsonify(temps)

In [20]:
# calculate TMIN, TAVG, TMAX with start and stop
results = session.query(*sel).\
       filter(measurement.date >= start).\
       filter(measurement.date <= end).all()
    
 # Unravel results into a 1D array and convert to a list
   temps = list(np.ravel(results))
   return jsonify(temps=temps)
if __name__ == '__main__':
   app.run()


IndentationError: unexpected indent (<ipython-input-20-0db1c3bddee5>, line 7)